In [11]:
import input_maker
import os # Needed for checking file existence and content
import numpy as np
import pandas as pd

In [14]:
# 2. Define your parameters
n_cham_bench = 154
m_emb = 2.8e-7 * np.ones(14) #solMass
m_pl = 2.8e-8 * np.ones(140) #solMass
m_cham_bench = np.concatenate((m_emb, m_pl))
cmf_uni = 0.33
output_file_example = "uniform_33.txt"

# 3. Call the function to generate the file
input_maker.generate_input_uniform(n_cham_bench, m_cham_bench, cmf_uni, output_file_example)

# 4. (Optional) Verify the file was created and inspect its content
print(f"\nChecking if '{output_file_example}' exists...")
if os.path.exists(output_file_example):
    print(f"File '{output_file_example}' found!")
    print("\nFirst 5 lines of the generated file:")
    with open(output_file_example, 'r') as f:
        for _ in range(6): # Read header + 5 data lines
            line = f.readline()
            if not line:
                break
            print(line.strip())
else:
    print(f"File '{output_file_example}' was NOT found.")

Generating 154 particles with CMF=0.33...
Output file: uniform_33.txt
Successfully generated 'uniform_33.txt' with 154 entries.

Checking if 'uniform_33.txt' exists...
File 'uniform_33.txt' found!

First 5 lines of the generated file:
1	2.80000000e-07	3.30000000e-01
2	2.80000000e-07	3.30000000e-01
3	2.80000000e-07	3.30000000e-01
4	2.80000000e-07	3.30000000e-01
5	2.80000000e-07	3.30000000e-01
6	2.80000000e-07	3.30000000e-01


# Step-function input example

In [15]:
#Ueda V3 example

percentages = [0.1, 0.15, 0.2, 0.25, 0.3]
bounds = [0.5, 0.6, 0.7, 0.8]
base_cmf = 0.33
semis_emb = np.loadtxt('semis_emb_chambers.txt')
semis_pl = np.loadtxt('semis_pl_chambers.txt')
semis_cham_bench = np.concatenate((semis_emb, semis_pl))

cmf_grid = pd.DataFrame(columns=['bounds', 'percentages', 'inner_cmf', 'outer_cmf'])
grid_index = 0

for bound in bounds:
    for perc in percentages:
        cmf1, cmf2 = input_maker.get_dual_cmf(m_cham_bench, [0], semis_cham_bench, [0], bound, perc, base_cmf)

        # Skip if any CMF > 1
        if cmf1 > 1 or cmf2 > 1:
            print(f"⚠️ Skipping: Bound = {bound}, perc. = {perc} → CMF > 1 (CMF_in = {cmf1:.4}, CMF_out = {cmf2:.4})")
            continue

        input_maker.generate_input_step_function(
            n_cham_bench, m_cham_bench, semis_cham_bench, cmf1, cmf2, bound,
            f"cham_bench_step_b{bound}_p{perc}.txt"
        )

        cmf_grid.loc[grid_index] = {
            'bounds': bound,
            'percentages': perc,
            'inner_cmf': cmf1,
            'outer_cmf': cmf2
        }

        print(f"Bound = {bound}, perc. of Fe trans. = {perc}, CMF_in = {cmf1:.4}, CMF_out = {cmf2:.4}")
        grid_index += 1

Output file: cham_bench_step_b0.5_p0.1.txt
Successfully generated 'cham_bench_step_b0.5_p0.1.txt' with 154 entries.
Bound = 0.5, perc. of Fe trans. = 0.1, CMF_in = 0.5402, CMF_out = 0.297
Output file: cham_bench_step_b0.5_p0.15.txt
Successfully generated 'cham_bench_step_b0.5_p0.15.txt' with 154 entries.
Bound = 0.5, perc. of Fe trans. = 0.15, CMF_in = 0.6452, CMF_out = 0.2805
Output file: cham_bench_step_b0.5_p0.2.txt
Successfully generated 'cham_bench_step_b0.5_p0.2.txt' with 154 entries.
Bound = 0.5, perc. of Fe trans. = 0.2, CMF_in = 0.7503, CMF_out = 0.264
Output file: cham_bench_step_b0.5_p0.25.txt
Successfully generated 'cham_bench_step_b0.5_p0.25.txt' with 154 entries.
Bound = 0.5, perc. of Fe trans. = 0.25, CMF_in = 0.8554, CMF_out = 0.2475
Output file: cham_bench_step_b0.5_p0.3.txt
Successfully generated 'cham_bench_step_b0.5_p0.3.txt' with 154 entries.
Bound = 0.5, perc. of Fe trans. = 0.3, CMF_in = 0.9605, CMF_out = 0.231
Output file: cham_bench_step_b0.6_p0.1.txt
Successfu

In [16]:
cmf_grid.to_csv("cham_bench_cmf_grid.csv", index=False)